## Setup model and dataset

In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

In [3]:
# setup model
class SimpleMLP(nn.Module):
    """
    简单的MLP模型
    """
    def __init__(self, input_dim, hidden_layers_dim=[64, 32, 8]):
        super().__init__()
        self.layers = []
        for i in range(len(hidden_layers_dim)):
            if i == 0: 
                self.layers.append(nn.Linear(input_dim, hidden_layers_dim[i]))
            else: 
                self.layers.append(nn.Linear(hidden_layers_dim[i-1], hidden_layers_dim[i]))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(hidden_layers_dim[-1], 1))
        self.layers = nn.Sequential(*self.layers)
        
    def forward(self, x):
        x = self.layers(x)  # [B, 1]
        x = x.squeeze(1)
        return x

In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def select_feat(train_data, valid_data, select_all=False):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid = train_data[:,:-1], valid_data[:,:-1]

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # [35:length - 1]
        feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], y_train, y_valid

In [5]:
# config
config = {
    'seed': 52,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.1,   # validation_size = train_size * valid_ratio
    'n_epochs': 1000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'weight_decay': 1e-5,  # L2 Regularization (weight decay).
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

In [6]:
same_seed(config['seed'])
train_data = pd.read_csv('datasets/covid/covid_train.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])
x_train, x_valid, y_train, y_valid = select_feat(train_data, valid_data, config['select_all'])
print(f"x_train size: {x_train.shape}, y_train size: {y_train.shape}")
train_dataset, valid_dataset = COVID19Dataset(x_train, y_train), COVID19Dataset(x_valid, y_valid)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)

x_train size: (2709, 53), y_train size: (2709,)


In [7]:
# train function
def train(num_epoch, model, optimizer, criterion, train_loader, val_loader, device, model_path, lr_scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epoch):
        train_loss = 0.0
        val_loss = 0.0
        
        # training
        model.train() # set the model to training mode
        for i, batch in enumerate(tqdm(train_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # validation
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                val_loss += loss.item()

        print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Loss: {train_loss/len(train_loader):3.5f} | Val loss: {val_loss/len(val_loader):3.5f}')

        # if the model improves, save a checkpoint at this epoch
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print(f'saving model with loss {val_loss/len(val_loader):.5f}')

        if lr_scheduler:
            lr_scheduler.step()

In [8]:
input_dim = x_train.shape[1]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Define loss as a function

In [9]:
def mse_loss(pred, target):
    return torch.mean((pred - target) ** 2)

In [11]:
criterion_1 = mse_loss
model = SimpleMLP(input_dim=input_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
train(100, model, optimizer, criterion_1, train_loader, valid_loader, device, 'models/simple_mlp.pt')

100%|██████████| 2/2 [00:00<00:00, 499.95it/s]


[001/100] Train Loss: 393.93820 | Val loss: 348.84509
saving model with loss 348.84509


100%|██████████| 2/2 [00:00<00:00, 499.92it/s]


[002/100] Train Loss: 391.97064 | Val loss: 416.79640


100%|██████████| 2/2 [00:00<00:00, 666.08it/s]


...

[098/100] Train Loss: 146.60638 | Val loss: 144.35190


100%|██████████| 2/2 [00:00<00:00, 399.88it/s]


[099/100] Train Loss: 145.18762 | Val loss: 143.07728


100%|██████████| 2/2 [00:00<00:00, 666.61it/s]

[100/100] Train Loss: 143.60540 | Val loss: 136.26231
saving model with loss 136.26231


## Define loss as a nn.Module

In [12]:
class MSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, target):
        return torch.mean((pred - target) ** 2)

In [14]:
criterion_2 = MSELoss()
model = SimpleMLP(input_dim=input_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
train(100, model, optimizer, criterion_2, train_loader, valid_loader, device, 'models/simple_mlp.pt')

100%|██████████| 2/2 [00:00<00:00, 400.07it/s]


[001/100] Train Loss: 415.25088 | Val loss: 355.43562
saving model with loss 355.43562


100%|██████████| 2/2 [00:00<00:00, 499.62it/s]


[002/100] Train Loss: 410.53878 | Val loss: 412.89938


100%|██████████| 2/2 [00:00<00:00, 641.13it/s]


...

[098/100] Train Loss: 126.71949 | Val loss: 125.84037


100%|██████████| 2/2 [00:00<00:00, 666.98it/s]


[099/100] Train Loss: 125.48237 | Val loss: 102.93153
saving model with loss 102.93153


100%|██████████| 2/2 [00:00<00:00, 499.77it/s]

[100/100] Train Loss: 124.54389 | Val loss: 129.30560
